# Prologue

<hr>

__Для подключения к API octoparts.com используется протокол OAuth2. Подключение осуществляется через оболочку nexar с использованием запросов на GraphQL. Для общения с этим сервисом я написал простой класс OctopartParser, файл с которым лежит в папке с этим ноутбуком. OctopartParser может получить токен от nexar, инициализировать сессию и выполнить некоторые базовые запросы к БД (получить данные о продавцах, категориях, отдельных деталях и т.д.). Nexar - это платный сервис с ограничением на 1000 бесплатных запросов. Поэтому я проведу анализ, например, тысячи первых микроконтроллеров, так как больших данных в связи с ограничением я получить не могу.__

In [128]:
import pandas as pd
import numpy as np

# created class OctoParser
import octopart_parcer
from octopart_parcer import OctopartParser
# created class to collect data from Yandex-disk
import data_preprocessing
from data_preprocessing import DataCollector

import re
from ast import literal_eval

import matplotlib.pyplot as plt
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

__Initializing the session__

In [ ]:
client_ID = '053e9f3c-85b2-4285-a1e3-fdf02d375b29'
client_secret = '05c3383d-46d9-4035-bbaf-2cc18635386d'

octopart_session = OctopartParser(client_id=client_ID, client_secret=client_secret)

In [ ]:
categories = octopart_session.get_all_categories(file_name='categories.csv')
sellers = octopart_session.get_all_sellers('sellers.csv')
manufacturers = octopart_session.get_all_manufacturers('manufacturers.csv')

In [ ]:
# take info about 1000 microcontrollers
parts = pd.DataFrame()
for _ in list(range(100, 1001, 100)):
    part = octopart_session.get_parts_by_query(q='microcontrollers', start=_, file_name=f'part_{_}.csv')
    parts = pd.concat([parts, part])

In [ ]:
parts = pd.DataFrame(parts.part.to_list())

<hr>

## Analisys


__I uploaded the parsed information to the yandex-disk, because I've reached the query-quantity limit on nexar and repeated requests are unavailable. But it is possible to create a code performing all ETL-cycle automatically if there is no query-limit.__

In [129]:
# download data
# _____________________________________________________________________________________________________________________________

In [130]:
parts = DataCollector().get_zip_yandex_disk('https://disk.yandex.ru/d/thq48EM_REwTCQ')\
                       .read_data()\
                       .concat_parts()\
                       .get_data()

zip file contains ['part_1000.csv', 'part_200.csv', 'part_300.csv', 'part_400.csv', 'part_500.csv', 'part_600.csv', 'part_700.csv', 'part_800.csv', 'part_900.csv', 'part_100.csv']


In [131]:
octopart_zip = DataCollector().get_zip_yandex_disk('https://disk.yandex.ru/d/93XVSDXU0PAZqw')\
                              .read_data()

zip file contains ['sellers.csv', 'categories.csv', 'manufacturers.csv']


In [132]:
sellers = octopart_zip.get_data(file_name='sellers')
categories = octopart_zip.get_data(file_name='categories')
manufacturers = octopart_zip.get_data(file_name='manufacturers')

In [133]:
# parts
# ______________________________________________________________________________________________________________________________

In [134]:
parts.head(3)

,Unnamed: 0,description,part
0,0,"ARM9® SAM9XE <em class=""highlight"">Microcontro...","{'category': {'id': '4299'}, 'estimatedFactory..."
1,1,"Add-On Board, Pic32 <em class=""highlight"">Micr...","{'category': {'id': '4267'}, 'estimatedFactory..."
2,2,"Precision Analog <em class=""highlight"">Microco...","{'category': {'id': '4300'}, 'estimatedFactory..."


In [135]:
# categories
# ______________________________________________________________________________________________________________________________

In [136]:
categories.head(3)

,Unnamed: 0,id,name,path,numParts,parentId,children,relevantAttributes
0,0,4161,Electronic Parts,/electronic-parts,24283237,NaN,"[{'id': '4393'}, {'id': '4569'}, {'id': '4364'...",[]
1,1,4393,Cables and Wire,/electronic-parts/cables-and-wire,647281,4161.0,"[{'id': '5031'}, {'id': '4394'}, {'id': '4399'...","[{'id': '712'}, {'id': '572'}, {'id': '275'}, ..."
2,2,5031,Audio / Video Cables,/electronic-parts/cables-and-wire/audio-video-...,7862,4393.0,[],"[{'id': '712'}, {'id': '737'}, {'id': '830'}, ..."


In [137]:
# manufaturers
# ______________________________________________________________________________________________________________________________

In [138]:
manufacturers.head()

,Unnamed: 0,id,name,homepageUrl,displayFlag,isDistributorApi,isVerified
0,0,12587,2-Power,https://2-power.com/,NaN,False,False
1,1,3739,2E Mechatronic,http://www.2e-mechatronic.de/,NaN,False,False
2,2,7674,2J,NaN,NaN,False,False
3,3,11375,2K Games,https://www.2k.com/,NaN,False,False
4,4,7042,2WIRE,NaN,NaN,False,False


__Need to transform a colnames and extract a dict-like data from the cells__

In [139]:
def transform_colnames(colname):
    re1 = r'[A-Z]{1}[a-z]+'
    re2 = r'[a-z]+'
    re3 = r'[A-Z]{2,}'
    
    colname_list = re.compile("%s|%s|%s" % (re1, re2, re3)).findall(colname)
    colname = ''
    for i in colname_list:
        if i.isupper():
            colname = colname + i + '_'
        else:        
            colname = colname + i.lower() + '_'
    return colname[:-1]

def get_info(df):
    info_df = pd.DataFrame()
    info_df['col_name'] = df.columns.to_list()
    info_df['dtype'] = df.dtypes.to_list()
    info_df['missing_values'] = df.isna().sum().to_list()
    info_df['unique_vals'] = df.nunique().to_list()
    info_df['total_vals'] = df.shape[0]
    return info_df



In [140]:
parts = pd.DataFrame(parts.part.apply(literal_eval).to_list())
parts.head(2)

,category,estimatedFactoryLeadDays,id,manufacturer,medianPrice1000,mpn,name
0,{'id': '4299'},112.0,77759171,{'id': '252'},"{'convertedPrice': 21.58, 'conversionRate': 1,...",AT91SAM9XE512B-CU,Microchip AT91SAM9XE512B-CU
1,{'id': '4267'},35.0,117437924,{'id': '3764'},"{'convertedPrice': 68, 'conversionRate': 1, 'c...",MIKROE-4591,mikroElektronika MIKROE-4591


In [141]:
# first of all, lets look data to figure out how much missing values it is
parts.isna().sum()

category                    17
estimatedFactoryLeadDays    47
id                           0
manufacturer                 0
medianPrice1000             13
mpn                          0
name                         0
dtype: int64

In [142]:
# remove missing values
parts = parts.dropna().reset_index(drop=True)
# extract dict-like info from median_price
parts = pd.concat([parts, pd.DataFrame(parts.medianPrice1000.to_list())], axis=1)
# extract category and manufacturer id
parts['category_id'] = parts.category.apply(pd.Series).astype(int)
parts['manufacturer_id'] = parts.category.apply(pd.Series).astype(int)
# drop unnecessary columns
parts = parts.drop(columns=['category', 'manufacturer','medianPrice1000'])
categories = categories.rename(columns={'id': 'category_id', 'name': 'category_name'})
manufacturers = manufacturers.rename(columns={'id': 'manufacturer_id', 'name': 'manufacturer_name'})


parts = pd.merge(left=parts, right=categories[['category_id', 'category_name']], on='category_id', how='inner')\
          .merge(manufacturers[['manufacturer_id', 'manufacturer_name']], on='manufacturer_id', how='inner')

# transform colnames
parts.columns = parts.columns.map(transform_colnames)

In [144]:
parts.head()

,estimated_factory_lead_days,id,mpn,name,converted_price,conversion_rate,converted_currency,currency,price,quantity,category_id,manufacturer_id,category_name,manufacturer_name
0,112.0,77759171,AT91SAM9XE512B-CU,Microchip AT91SAM9XE512B-CU,21.580000,1,USD,USD,21.580000,1000,4299,4299,Microprocessors,Video Display Corporation
1,182.0,33923139,LPC1768FET100Y,NXP Semiconductors LPC1768FET100Y,11.502920,1,USD,USD,11.502920,1000,4299,4299,Microprocessors,Video Display Corporation
2,182.0,10459874,"LPC2460FBD208,551","NXP Semiconductors LPC2460FBD208,551",12.875800,1,USD,USD,12.875800,1000,4299,4299,Microprocessors,Video Display Corporation
3,364.0,19769337,"LPC1224FBD48/121,1","NXP Semiconductors LPC1224FBD48/121,1",4.563585,1,USD,USD,4.563585,1000,4299,4299,Microprocessors,Video Display Corporation
4,182.0,21325715,"LPC11E13FBD48/301,","NXP Semiconductors LPC11E13FBD48/301,",2.811490,1,USD,USD,2.811490,1000,4299,4299,Microprocessors,Video Display Corporation
